In [302]:
import folium
import pandas as pd
import geopandas as gpd

from geopy.geocoders import Nominatim
from shapely.ops import unary_union
from shapely import wkt
from tqdm import tqdm

In [303]:
locations_df = pd.read_csv('../data/references/handmade/qc-population-centers.csv')
locations_df.head()

,Location,City,RCM,Region,Bounding Territory,Display Name,Bounding Type,Bounding Population,GeoPy Index
0,Rimouski,Rimouski,Rimouski-Neigette,Bas-Saint-Laurent,Bas-Saint-Laurent,Bas-Saint-Laurent,Administative Region,200000,0
1,Gaspé,Gaspé,La Côte-de-Gaspé,Gaspésie-Îles-de-la-Madeleine,Gaspésie-Îles-de-la-Madeleine,Gaspésie-Îles-de-la-Madeleine,Administative Region,90000,0
2,La Cité-Limoilou,Québec,Québec,Capitale-Nationale,La Cité-Limoilou,La Cité-Limoilou,Québec City Borough,100000,0
3,Les Rivières,Québec,Québec,Capitale-Nationale,Les Rivières,Les Rivières,Québec City Borough,75000,0
4,Sainte-Foy-Sillery-Cap-Rouge,Québec,Québec,Capitale-Nationale,Sainte-Foy-Sillery-Cap-Rouge,Sainte-Foy-Sillery-Cap-Rouge,Québec City Borough,100000,0


In [304]:
geolocator = Nominatim(user_agent='housing-qc-viz')
qc_chloro_map = folium.Map(location=[47.0, -70.5], zoom_start=8)

In [305]:
geocode = geolocator.geocode("Deux-Montagnes" + ", Qc", geometry='wkt', exactly_one=False)
geocode

[Location(Deux-Montagnes, Laurentides, Québec, Canada, (45.5372581, -73.9043321, 0.0)),
 Location(Deux-Montagnes, Laurentides, Québec, Canada, (45.5255369, -73.9981516, 0.0)),
 Location(Deux-Montagnes, Autoroute 640, St-Eustache, Deux-Montagnes, Laurentides, Québec, J7P 1Z8, Canada, (45.5453988, -73.9207628, 0.0)),
 Location(Deux-Montagnes, Autoroute 640, Sainte-Marthe-sur-le-Lac, Deux-Montagnes, Laurentides, Québec, J7R 6B2, Canada, (45.5429439, -73.9273496, 0.0)),
 Location(Deux-Montagnes, Boulevard de Deux-Montagnes, Deux-Montagnes, Laurentides, Québec, J7R 6B2, Canada, (45.5456833, -73.9124135, 0.0)),
 Location(Deux-Montagnes, Boulevard de Deux-Montagnes, Deux-Montagnes, Laurentides, Québec, J7R 6B2, Canada, (45.545667550000005, -73.91237803531664, 0.0))]

In [306]:
print("Unique locations: " + str(len(locations_df['Display Name'].unique())))

Unique locations: 107


In [307]:
def get_polygons(location_string: str, geopy_index: int):
    polygons = []
    substracts = []
    for location in location_string.replace("[", "").replace("]", "").split(","):
        if location.startswith("-"):
            geocodes = geolocator.geocode(location[1:] + ', QC', geometry='wkt', exactly_one=False)
            for geocode in geocodes:
                substracts.append(wkt.loads(geocode.raw['geotext']))
        else:
            geocode = geolocator.geocode(location + ', QC', geometry='wkt', exactly_one=False)[geopy_index]
            polygons.append(wkt.loads(geocode.raw['geotext']))

    return polygons, substracts
        

In [308]:
def add_geometry(location_string: str, popup: str, geopy_index: int):
    polygons, substracts = get_polygons(location_string, geopy_index)
    geo_json = gpd.GeoSeries(unary_union(polygons)).simplify(tolerance=0.001).to_json()
    geo_json = gpd.GeoSeries(unary_union(polygons).difference(unary_union(substracts))).simplify(tolerance=0.001).to_json()
    geo_j = folium.GeoJson(data=geo_json, style_function=lambda x: {'fillColor': 'gray', 'fillOpacity': 0.4})
    folium.Popup(popup).add_to(geo_j)
    geo_j.add_to(qc_chloro_map)

In [309]:
dic = {"Mirabel": 0, "Deux-Montagnes": 0, "Thérèse-De Blainville": 0, "La Rivière-du-Nord": 0}
for i in tqdm(['[Mirabel,-Deux-Montagnes,-La Rivière-du-Nord]', 'Deux-Montagnes', 'La Rivière-du-Nord'], desc="Adding Geometries"):
    try:
        print(i)
        add_geometry(i, i, 0)
    except:
        print("Unable to calculate geometry for: " + i)

Adding Geometries:   0%|          | 0/3 [00:00<?, ?it/s]

[Mirabel,-Deux-Montagnes,-La Rivière-du-Nord]


Adding Geometries:  33%|███▎      | 1/3 [00:01<00:02,  1.33s/it]

poll
Deux-Montagnes


Adding Geometries:  67%|██████▋   | 2/3 [00:01<00:00,  1.14it/s]

poll
La Rivière-du-Nord


Adding Geometries: 100%|██████████| 3/3 [00:02<00:00,  1.25it/s]

poll


In [310]:
qc_chloro_map

In [8]:
for i, r in tqdm(locations_df.iterrows(), desc="Adding Geometries", total=locations_df.shape[0]):
    try:
        add_geometry(r["Bounding Territory"], r["Display Name"], r["GeoPy Index"])
    except:
        print("Unable to calculate geometry for: " + r["Bounding Territory"])


Adding Geometries:   6%|▌         | 8/138 [00:03<01:02,  2.07it/s]

Unable to calculate geometry for: La Haute-Saint-Charles


Adding Geometries:  15%|█▌        | 21/138 [00:15<02:42,  1.39s/it]

In [ ]:
qc_chloro_map

In [ ]:
##################

In [1]:
geocode = geolocator.geocode("Mirabel" + ", Qc", geometry='wkt', exactly_one=False)
geocode

NameError: name 'geolocator' is not defined

In [ ]:
geocode = geolocator.geocode("Senneville" + ', QC', geometry='wkt')
wkt_shape = wkt.loads(geocode.raw['geotext'])
geo_json = gpd.GeoSeries(wkt_shape).simplify(tolerance=0.001).to_json()

In [ ]:
folium.GeoJson(data=geo_json, style_function=lambda x: {'fillColor': 'orange'}).add_to(qc_chloro_map)

In [ ]:
qc_chloro_map